In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from Metrics_v2 import StrokeMetrics, ALSMetrics, ALSorStrokeMetrics
import pandas as pd
import numpy as np
import argparse
import os
from typing import Dict, List

In [3]:
# Some helper functions

# Audio tasks use an audio.Table while others use a video.Table
audio_tasks = ["BBP_NORMAL"]

# Used to extract the patient type from the landmark file name
ids = {
    "stroke": ["OP", "S"],
    "healthy": ["N"],
    "als": ["A"]
}

# Defines which metric calculator to use based on the 'metric_type' argument
metric_calc_map = {
    "stroke": StrokeMetrics,
    "als": ALSMetrics,
    "all": ALSorStrokeMetrics
}


def compute_metrics(inputs, out_path, metrics_type = 'all'):
    """
    Computes the metrics for every file in the data csv
    :param inputs: A dataframe with columns: csv_path, rep_{}_start,
                                                       rep_{}_end
    :param out_path: A path to a csv to store the metric data in
    """
    # Key is task name and value is all repetitions of the task
    metric_frames: Dict[str, List[pd.DataFrame]] = {}
    for index, row in inputs.iterrows():
        # Read all data from the input dataframe
        try:
            data_path = row['csv_path']
            rest_path = row['rest_path']
            data_frame = pd.read_csv(data_path)
            rest_frame = pd.read_csv(rest_path)
            task_type = row['task']
            subject_id = row['subject']
            subject_type = row['type']
        except KeyError:
            raise KeyError("Input csv must contain 'csv_path', 'rest_path', 'rest_table_path' \
            'task', 'subject', and 'type' columns. \nFurthermore, 'csv_path' \
            and 'rest_path' must be files on the disk.")
        print("Starting:", data_path, "with", subject_type, "patients")
        rep_ranges = []
        rep_i = 1
        # Grab data about the start and end of repetitions
        while f"rep_{rep_i}_start" in row and f"rep_{rep_i}_end" in row:
            try:
                start_val = row[f"rep_{rep_i}_start"]
                end_val = row[f"rep_{rep_i}_end"]
                rep_ranges.append(range(int(start_val), int(end_val)))
            except ValueError as e:
                pass
            rep_i += 1
        rep_i -= 1

        # Calculate metrics
        metric_calc = metric_calc_map[metrics_type](data_frame, rest_frame)
        # metric_calc = StrokeMetrics(data_frame, rest_frame)
        metric_frame = None
        for rep_range in rep_ranges:
            metrics = metric_calc.compute_metrics(active_frames=rep_range)
            if metric_frame is None:
                metric_frame = metrics
            else:
                metric_frame = pd.concat([metric_frame, metrics], ignore_index=True)
        rep_nums = list(range(1, rep_i+1))

        # Insert metadata to final csv
        metric_frame.insert(0, 'rep', pd.Series(rep_nums))
        metric_frame.insert(0, 'type', pd.Series(np.full(rep_i, int(not(subject_type == "healthy")))))
        metric_frame.insert(0, 'task', pd.Series(np.full(rep_i, task_type)))
        metric_frame.insert(0, 'subject_id', pd.Series(np.full(rep_i, subject_id)))
        try:
            metric_frames[task_type].append(metric_frame)
        except KeyError:
            metric_frames[task_type] = [metric_frame]

    # save data to disk
    for task, metrics in metric_frames.items():
        csv_name = "metric_output_{}.csv".format(task)
        stand_csv_name = "metric_output_{}_standardized.csv".format(task)
        all_metrics_frame = pd.concat(metrics, ignore_index=True)

        # Create a new frame standardized by z-score of columns
        all_metrics_stand = all_metrics_frame.copy()
        for col in all_metrics_stand.columns:
            if col in ['subject_id', 'type', 'task', 'rep']:
                continue
            feature = all_metrics_stand[col]
            mean = np.mean(feature)
            sd = np.std(feature)
            if sd != 0:
                z_scores = (feature - mean) / sd
            else:
                z_scores = ["NA" for i in range(len(feature))]
            all_metrics_stand[col] = z_scores
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        all_metrics_frame.to_csv(os.path.join(out_path, csv_name))
        all_metrics_stand.to_csv(os.path.join(out_path, stand_csv_name))
    print(f"Saved metrics to {os.path.abspath(out_path)}")
    return metric_frames


def convert_time_to_frames(time_file, landmarks_file):
    """
    Converts audio timestamps into video frames
    :param time_file:
    :param landmarks_file:
    :return:
    """
    data = pd.read_csv(time_file, index_col=None)
    landmarks_DF = pd.read_csv(landmarks_file, index_col=0)
    tmin = data[data.columns[0]].values
    tmax = data[data.columns[-1]].values
    vid_time = landmarks_DF['Time_Stamp (s)'].values[1:]
    positions = np.zeros((len(tmin), 2), dtype=int)
    for k, (mini, maxi) in enumerate(zip(tmin, tmax)):
        positions[k, 0] = int(np.argmin(abs(vid_time - mini)))
        positions[k, 1] = int(np.argmin(abs(vid_time - maxi)))
    new_df = pd.DataFrame()
    new_df['Tstart'] = positions[:, 0]
    new_df['text'] = data['text']
    new_df['Tend'] = positions[:, 1]
    return new_df

# Main Program 
You need to provide a .csv file with three columns : 'landmarks'  'landmarks_table'   'rest' 


- the 'landmarks' column contains the path to the .csv file that contains the landmark positions for the task under analysis (either 2d or 3d)
- the 'landmarks_table' column cotains the path to the .table file that contains the information about the repetitions either as frames 
  (_video.table) or as time stamps (_audio.table). When using time stamps, there has to be a 'FrameInfoColor.csv' file that contains the 
  relation between time stamps and frame number
- The 'rest' column contains the path to the .csv file that contains the landmark positions for the REST task (either 2d or 3d)


NOTE: There has to be a 'REST' file associated with the subject 
### The program won't continue if the REST file is not provided

In [4]:

def save_results(input_csv, output_folder, metric_type):
    analysis_csv = pd.DataFrame()
    files = pd.read_csv(input_csv)
    for i, row in files.iterrows():
        # Construct a dataframe with the information necessary for processing
        landmark_path = os.path.abspath(row["landmarks"])
        landmark_file_name = os.path.basename(landmark_path)
        table_path = os.path.abspath(row["landmarks_table"])
        table_file_name = os.path.basename(table_path)
        rest_path = os.path.abspath(row["rest"])
        rest_file_name = os.path.basename(table_path)
        print(landmark_path)
        print()
        print(rest_path)
        print()
        print(table_path)

        if not os.path.isfile(landmark_path):
            raise RuntimeError("Landmark path is not a file on disk")
        if not os.path.isfile(rest_path):
            raise RuntimeError("Rest file is not a file on disk")
        if not os.path.isfile(table_path):
            raise RuntimeError("Table file is not a file on disk")

        landmark_file_data = landmark_file_name.split("_")
        subject = landmark_file_data[0]
        subject_type = ""
        # Check if a user fits into a known subject type
        for s_type, prefixes in ids.items():
            for prefix in prefixes:
                if prefix in subject:
                    subject_type = s_type
        task = "_".join(landmark_file_data[2:4])
        parsed_data = pd.DataFrame(columns=["csv_path", "type", "rest_path", "subject", "task"])
        parsed_data.loc[0] = 0

        # If it is an audio task, convert the table file to frame numbers
        if 'audio' in table_file_name:
            video_info = convert_time_to_frames(table_path, landmark_path)
        else:
            video_info = pd.read_csv(table_path)


        parsed_data["csv_path"] = landmark_path
        parsed_data["rest_path"] = rest_path
        parsed_data["type"] = subject_type
        parsed_data["subject"] = subject
        parsed_data["task"] = task
        for index, row in video_info.iterrows():
            parsed_data[f"rep_{index+1}_start"] = row[video_info.columns[0]]
            parsed_data[f"rep_{index+1}_end"] = row[video_info.columns[-1]] 
        analysis_csv = analysis_csv.append(parsed_data, ignore_index=True, sort=False)




    compute_metrics(analysis_csv, output_folder,metric_type);

In [13]:
input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv"

df= pd.read_csv(input_csv)
df.head()

,landmarks,landmarks_table,rest
0,E:\FaceData_\video_data\ALS\old\A001\landmarks...,E:\FaceData_\video_data\ALS\old\A001\A001_02_B...,E:\FaceData_\video_data\ALS\old\A001\landmarks...
1,E:\FaceData_\video_data\ALS\old\A001\landmarks...,E:\FaceData_\video_data\ALS\old\A001\A001_02_N...,E:\FaceData_\video_data\ALS\old\A001\landmarks...
2,E:\FaceData_\video_data\ALS\old\A001\landmarks...,E:\FaceData_\video_data\ALS\old\A001\A001_02_N...,E:\FaceData_\video_data\ALS\old\A001\landmarks...
3,E:\FaceData_\video_data\ALS\old\A004\landmarks...,E:\FaceData_\video_data\ALS\old\A004\A004_02_B...,E:\FaceData_\video_data\ALS\old\A004\landmarks...
4,E:\FaceData_\video_data\ALS\old\A004\landmarks...,E:\FaceData_\video_data\ALS\old\A004\A004_02_N...,E:\FaceData_\video_data\ALS\old\A004\landmarks...


In [14]:
def change_path(df_cell):

    df_cell = df_cell.replace("E:\FaceData_", "\\Users\denizjafari\documents\CODE\ClinicalScore\FaceData_")
    df_cell = df_cell.replace("\\", "/")
    return df_cell

In [15]:
# change the path to the correct path on my MAC 
try:
    df['landmarks'] = df['landmarks'].apply(lambda x: change_path(x))
    df['landmarks_table'] = df['landmarks_table'].apply(lambda x: change_path(x))
    df['rest'] = df['rest'].apply(lambda x: change_path(x))
except:
    pass

# Write the results into csv
df.to_csv('ALS_New_Table.csv')
try:
    df = df.drop(['Unnamed: 0'], axis=1)
except:
    pass

df.head()

,landmarks,landmarks_table,rest
0,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
1,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
2,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
3,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
4,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...


In [16]:
metric_type = 'all' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#finetuned
input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)



/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/A001_02_BBP_NORMAL_audio.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_NSM_OPEN_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/A001_02_NSM_OPEN_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_NSM_SPREAD_landmarksFiltered3D.csv

/Users/denizjafari/documents/COD

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A013_02_BBP_NORMAL_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A013_02_RST_REST3_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/A013_02_BBP_NORMAL_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A013_02_NSM_BIGSMILE_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A013_02_RST_REST3_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A013_02_NSM_SPREAD_landmarksFiltered3D.csv

/Users/denizjafari/doc

Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_NSM_OPEN_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_NSM_SPREAD_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A004/landmarks_finetuned_w15/A004_02_BBP_NORMAL_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A004/landmarks_finetuned_w15/A004_02_NSM_OPEN_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A004/landmarks_finetuned_w15/A00

/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/Metrics_v2.py:369: RuntimeWarning: invalid value encountered in greater
  return z_scores > z_threshold


Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A005/landmarks_finetuned_w15/A005_02_NSM_OPEN_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A005/landmarks_finetuned_w15/A005_02_NSM_SPREAD_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_BBP_NORMAL_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_NSM_OPEN_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_NSM_SPREAD_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A01

In [9]:
break 

SyntaxError: 'break' outside loop (<ipython-input-9-a17de20832ec>, line 1)

In [ ]:
### THE END!

In [ ]:
#finetuned
input_csv = r"E:\FaceData_\results\Stroke_vs_Controls_new_finetuned.csv" #csv file with landmarks and .TABLE info
output_folder = r"E:\FaceData_\results\Stroke_vs_Controls_new_finetuned" #where to store the results
save_results(input_csv, output_folder, metric_type)


In [ ]:
metric_type = 'stroke' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'

#pretrained
input_csv = r"E:\FaceData_\results\Stroke_vs_Controls_database_pretrained.csv" #csv file with landmarks and .TABLE info
output_folder = r"E:\FaceData_\results\Stroke_vs_Controls_database_pretrained" #where to store the results
save_results(input_csv, output_folder, metric_type)


#finetuned
input_csv = r"E:\FaceData_\results\Stroke_vs_Controls_database_finetuned.csv" #csv file with landmarks and .TABLE info
output_folder = r"E:\FaceData_\results\Stroke_vs_Controls_database_finetuned" #where to store the results
save_results(input_csv, output_folder, metric_type)